In [1]:
from google.colab import files
uploaded = files.upload()

Saving bank.csv to bank.csv


In [4]:
df = pd.read_csv("bank.csv", sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve


In [5]:
df["y"]=df["y"].map({"yes":1,"no":0})

In [7]:
X=df.drop(columns=["y"])
y=df["y"]

In [8]:
categorical_cols = X.select_dtypes(include="object").columns
numerical_cols = X.select_dtypes(exclude="object").columns

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numerical_cols)
    ]
)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [12]:
model = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("classifier", LogisticRegression(max_iter=1000))
])

In [13]:
model.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')),
                                                 ('num', 'passthrough',
                                                  Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object'))])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [16]:
y_prob = model.predict_proba(X_test)[:, 1]

In [17]:
y_pred_05 = (y_prob >= 0.5).astype(int)

In [18]:
cm = confusion_matrix(y_test, y_pred_05)
print(cm)

[[972  29]
 [ 91  39]]


In [20]:
#--sensitivy
precision = precision_score(y_test, y_pred_05)
recall = recall_score(y_test, y_pred_05)
f1 = f1_score(y_test, y_pred_05)
specificity = cm[0,0] / (cm[0,0] + cm[0,1])

print("Precision:", precision)
print("Recall (Sensitivity):", recall)
print("Specificity:", specificity)
print("F1-score:", f1)

Precision: 0.5735294117647058
Recall (Sensitivity): 0.3
Specificity: 0.971028971028971
F1-score: 0.3939393939393939


In [21]:
roc_auc = roc_auc_score(y_test, y_prob)
print("ROC-AUC:", roc_auc)

ROC-AUC: 0.8861215707369553


In [22]:
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
youden_index = tpr - fpr
best_threshold = thresholds[np.argmax(youden_index)]

print("Optimized Threshold:", best_threshold)

Optimized Threshold: 0.09993711784011383


In [23]:
y_pred_opt = (y_prob >= best_threshold).astype(int)
cm_opt = confusion_matrix(y_test, y_pred_opt)
precision_opt = precision_score(y_test, y_pred_opt)
recall_opt = recall_score(y_test, y_pred_opt)
f1_opt = f1_score(y_test, y_pred_opt)
print(cm_opt)
print("Precision:", precision_opt)
print("Recall:", recall_opt)
print("F1-score:", f1_opt)

[[796 205]
 [ 22 108]]
Precision: 0.3450479233226837
Recall: 0.8307692307692308
F1-score: 0.48758465011286684


In [25]:
prob_df = pd.DataFrame({
    "RecordId": y_test.index,
    "Probability(yes)": y_prob,
    "PredictedLabel": y_pred_opt
})
prob_df.to_csv("probabilities.csv", index=False)

In [26]:
import os
os.listdir()

['.config', 'probabilities.csv', 'bank.csv', 'sample_data']

In [27]:
prob_df.to_csv("/content/probabilities.csv", index=False)
print("File saved successfully")

File saved successfully


In [29]:
from google.colab import files
files.download("probabilities.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>